In [1]:
import pymc as pm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import arviz as az
from sklearn.preprocessing import StandardScaler, LabelEncoder


In [7]:
import pymc as pm
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, mutual_info_classif

def bayesian_logistic_regression(X, y):
    # Data preparation
    X_std = X.copy()
    # Convert to numpy arrays
    X_data = X_std.values
    y_data = y.values

    # Bayesian Logistic Regression Model
    with pm.Model() as model:
        # Priors
        β = pm.Normal('β', mu=0, sigma=10, shape=X_data.shape[1])
        α = pm.Normal('α', mu=0, sigma=10)

        # Linear predictor
        logit_p = α + pm.math.dot(X_data, β)
        p = pm.Deterministic('p', pm.math.sigmoid(logit_p))

        # Likelihood
        y_obs = pm.Bernoulli('y_obs', p=p, observed=y_data)

        # Sampling
        trace = pm.sample(2000, tune=1000, cores=2)

    return trace, X_std.columns

def analyze_results(trace, feature_names):
    """Analyze and visualize Bayesian results"""
    # Summary statistics
    summary = az.summary(trace, var_names=['β'], hdi_prob=0.95)
    summary['feature'] = feature_names
    summary = summary.set_index('feature')

    # Calculate odds ratios
    summary['odds_ratio'] = np.exp(summary['mean'])
    summary['pct_effect'] = (summary['odds_ratio'] - 1) * 100

    # Classify factors
    summary['factor_type'] = np.where(
        summary['mean'] > 0,
        'Risk',
        'Protective'
    )

    # Visualize
    plt.figure(figsize=(12, 8))
    sns.barplot(
        x='pct_effect',
        y=summary.index,
        data=summary,
        hue='factor_type',
        palette={'Risk': 'red', 'Protective': 'green'},
        dodge=False
    )
    plt.axvline(0, color='black', linestyle='--')
    plt.title('Percentage Effect on Churn Probability')
    plt.xlabel('Percentage Effect')
    plt.ylabel('Feature')
    plt.tight_layout()
    plt.savefig('factor_analysis.png')
    plt.close()

    return summary



In [3]:
# importing google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Load your data
df = pd.read_parquet("./drive/MyDrive/churn_prediction/telco_customer_churn_preprocessed-train-parametric_models.parquet")
X = df.drop(columns=["Churn"])
y = df["Churn"]

In [8]:
trace, features_names = bayesian_logistic_regression(X, y)

Output()

In [14]:
# 1. Save results as pickle
import pickle
def save_bayesian_results(trace, feature_names, file_path):
    """Save Bayesian analysis results as pickle file"""
    results = {
        'trace': trace,
        'feature_names': feature_names
    }
    with open(file_path, 'wb') as f:
        pickle.dump(results, f)
    print(f"Results saved to {file_path}")
save_bayesian_results(trace, features_names, './drive/MyDrive/churn_prediction/bayesian_results.pkl')


Results saved to ./drive/MyDrive/churn_prediction/bayesian_results.pkl


In [16]:
# 2. Create odds ratio and effect size dataframe
def compute_odds_effects(trace, feature_names):
    """
    Compute odds ratios and percentage effects from Bayesian trace

    Returns:
        DataFrame with columns:
        feature, coef_mean, coef_hdi_low, coef_hdi_high,
        odds_ratio, pct_effect, effect_type
    """
    # Extract coefficient samples
    β_samples = trace.posterior['β'].values

    # Calculate statistics
    results = []
    for i, feature in enumerate(feature_names):
        # Coefficient statistics
        coef_vals = β_samples[:, :, i].flatten()
        coef_mean = np.mean(coef_vals)
        hdi = az.hdi(coef_vals, hdi_prob=0.95)

        # Odds ratio and percentage effect
        odds_ratio = np.exp(coef_mean)
        pct_effect = (odds_ratio - 1) * 100

        # Determine effect type
        effect_type = "Risk" if coef_mean > 0 else "Protective"

        results.append({
            'feature': feature,
            'coef_mean': coef_mean,
            'coef_hdi_low': hdi[0],
            'coef_hdi_high': hdi[1],
            'odds_ratio': odds_ratio,
            'pct_effect': pct_effect,
            'effect_type': effect_type
        })

    # Create and sort dataframe
    df = pd.DataFrame(results)
    df = df.sort_values('pct_effect', key=abs, ascending=False)

    return df
# 2. Compute odds effects
odds_effects_df = compute_odds_effects(trace, features_names)

# Save to CSV
odds_effects_df.to_csv('./drive/MyDrive/churn_prediction/feature_odds_effects.csv', index=False)


The odds ratio and percentage effect of each feature are calculated as shown below:

In [19]:
odds_effects_df.sort_values('coef_mean', ascending=False)

,feature,coef_mean,coef_hdi_low,coef_hdi_high,odds_ratio,pct_effect,effect_type
4,PaperlessBilling,0.151145,0.017234,0.286330,1.163166,16.316564,Risk
0,SeniorCitizen,-0.092348,-0.261096,0.085139,0.911788,-8.821219,Protective
1,Partner,-0.203472,-0.349687,-0.063944,0.815893,-18.410677,Protective
2,Dependents,-0.556230,-0.741535,-0.390344,0.573367,-42.663320,Protective
3,tenure,-0.859111,-0.975315,-0.746163,0.423538,-57.646155,Protective
5,MonthlyCharges,-1.443808,-3.287936,0.407508,0.236027,-76.397272,Protective
27,InternetService_Fiber optic,-2.908777,-6.417150,-0.051586,0.054542,-94.545763,Protective
24,StreamingMovies_Yes,-4.431846,-7.428639,-1.850927,0.011893,-98.810748,Protective
23,StreamingTV_Yes,-4.561930,-7.633843,-2.080528,0.010442,-98.955811,Protective
26,StreamingMovies_No,-5.165156,-8.137350,-2.569536,0.005712,-99.428783,Protective


The features selected for segmentation:

In [42]:
Segmentation_Features = ['SeniorCitizen', 'Dependents', 'MultipleLines_Yes','InternetService_Fiber optic','OnlineBackup_Yes','OnlineSecurity_Yes', 'DeviceProtection_Yes','TechSupport_Yes','PaperlessBilling','Contract_Month-to-month','tenure','MonthlyCharges' ]